In [1]:
import os
import numpy as np
import pandas as pd

## Считываем набор данных

В прикладных задачах машинного обучения очень важен процесс извлечения признаков (feature extraction), в ходе которого данные интерпретируются в информативные признаки. Также этот процесс может называться проектирование признаков (feature engineering), это весьма трудоемкая и творческая задача. В рамках работы мы опустим эту часть и поспользуемся предобработанными данными. 

In [99]:
def read_data(path, filename):
    return pd.read_csv(os.path.join(path, filename))

df = read_data('/data/notebook_files', 'train.csv')
df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


Теперь, загрузим полный набор данных и сохранить его под следующими четырьмя переменными:

* `train_X`: признаки, используемые для обучения модели
* `train_y`: метки, используемые для обучения модели
* `test_X`: признаки, используемые для проверки модели
* `test_y`: метки, используемые для проверки модели

In [100]:
def load_dataset(label_dict):
    train_X = read_data('/data/notebook_files', 'train.csv').values[:,:-2]
    train_y = read_data('/data/notebook_files', 'train.csv')['Activity']
    train_y = train_y.map(label_dict).values
    test_X = read_data('/data/notebook_files', 'test.csv').values[:,:-2]
    test_y = read_data('/data/notebook_files', 'test.csv')
    test_y = test_y['Activity'].map(label_dict).values
    return (train_X, train_y, test_X, test_y)
label_dict = {'WALKING':0, 'WALKING_UPSTAIRS':1, 'WALKING_DOWNSTAIRS':2, 'SITTING':3, 'STANDING':4, 'LAYING':5}
train_X, train_y, test_X, test_y = load_dataset(label_dict)

## Выбор модели 1 (KNeighborsClassifier)
Импортируйте выбранную вами модель из библиотеки `sklearn` и инициализируйте её в объект `model`:

In [101]:
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier(n_neighbors = 3)    

## Для подбора наилучшего гиперпараметра(n_neighbors) воспользуемся GridSearchCV
Для нашей модели мы укажем диапазон значений для «n_neighbors», чтобы увидеть, какое значение лучше всего подходит для нашей модели. Для этого мы создадим словарь, установив в качестве ключа «n_neighbors» и используя numpy для создания массива значений от 1 до 24.

In [102]:
from sklearn.model_selection import GridSearchCV  
param_grid = {'n_neighbors' : np.arange(1, 25)} # create a dictionary of all values we want to test for n_neighbors
knn_gscv = GridSearchCV(model1, param_grid, cv = 5) #use gridsearch to test all values for n_neighbors, cv = кол-во разрезов выборки
knn_gscv.fit(train_X, train_y)  #fit model to data

#check top performing n_neighbors value
knn_gscv.best_params_

{'n_neighbors': 16}

### Мы выяснили, что наилучший параметр для n_neighbors это 16. Используем эту информацию для обучения модели и получения метрик

Опишите вашу мотивацию по выбору модели.

Алгоритм может быть применим к выборкам с большим количеством атрибутов, как в нашем случае, поэтому я счел этот алгоритм подходящим. К тому же, этот алгоритм довольно прост в реализации.

## Обучение модели

Обучите модель, используя признаки из обучающего набора (`train_X`) и метки в качестве базовой истины (`train_y`).

In [103]:
model1_best = KNeighborsClassifier(n_neighbors = 16 )
model1_best.fit(train_X, train_y)

KNeighborsClassifier(n_neighbors=16)

## Оценка модели
Используйте обученную модель для прогнозирования активности движения, используя признаки из тестового набора (`test_X`). Прогнозы сохраните в списке `yhat`.

In [104]:
yhat = model1_best.predict(test_X)
yhat

### Выведите отчет о классификации, сравнив предсказания (`yhat`) с базовой истиной (`test_y`).

### В чем разница между показателями `precision` и `recall`? Что такое показатель `F1`?


Ответ:

Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными, а recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.

Recall демонстрирует способность алгоритма обнаруживать данный класс, а precision — способность отличать этот класс от других классов.

F-мера (в общем случае $\ F_\beta$) — среднее гармоническое precision и recall. 
F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.
F1 = 2(recall * precision)/(recall + precision)

In [105]:
from sklearn.metrics import classification_report
target_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

print(classification_report(test_y, yhat, target_names=target_names))

                    precision    recall  f1-score   support

           Walking       0.84      0.99      0.91       496
  Walking Upstairs       0.89      0.90      0.90       471
Walking Downstairs       0.96      0.77      0.85       420
           Sitting       0.92      0.82      0.86       491
          Standing       0.85      0.93      0.89       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.91      2947
         macro avg       0.91      0.90      0.90      2947
      weighted avg       0.91      0.91      0.90      2947



## Вывод KNeighborsClassifier
 Показатели precision и recall показывают, что алгоритм довольно точно предсказал действия человека. Изначально я не знал, какой гиперпараметр использовать в качестве n_neighbors. После поиска возможных решений в интернете я узнал о GridSearchCV. Оказалось, что лучшие метрики достигаются при n_neighbors = 16.

## Выбор модели 2 (GaussianNB)
Импортируйте выбранную вами модель из библиотеки `sklearn` и инициализируйте её в объект `model`:

In [106]:
from sklearn.naive_bayes import GaussianNB


model2 = GaussianNB()
model2.fit(train_X, train_y)

yhat = model2.predict(test_X)
yhat

In [107]:
print(classification_report(test_y, yhat, target_names=target_names))

                    precision    recall  f1-score   support

           Walking       0.82      0.84      0.83       496
  Walking Upstairs       0.76      0.96      0.84       471
Walking Downstairs       0.83      0.61      0.70       420
           Sitting       0.58      0.75      0.65       491
          Standing       0.80      0.86      0.83       532
            Laying       0.96      0.60      0.74       537

          accuracy                           0.77      2947
         macro avg       0.79      0.77      0.77      2947
      weighted avg       0.79      0.77      0.77      2947



## Вывод GaussianNB
 Показатели precision и recall показывают, что алгоритм не очень точно предсказал действия человека. Более того, для разных действий показатели могут сильно розниться(например, precision для Laying 0.96, в то время как для Sitting всего 0.58, что является довольно низким показателем). F1 мера говорит о том, что этот алгоритм классификации не является оптимальным для нашей задачи. Возможно, это связано с тем, что действия человека не являются полностью статистически независимыми, что является важным аспектом для Наивного Байесовского алгоритма.  

### Можно рассмотреть матрицу неточностей для понимания того, какие именно классы чаще всего путал наш алгоритм между собой.

In [108]:
print(confusion_matrix(test_y,yhat))

[[416  38  42   0   0   0]
 [  9 451  11   0   0   0]
 [ 80  83 257   0   0   0]
 [  0   7   0 368 111   5]
 [  0  15   0  54 455   8]
 [  0   3   0 211   0 323]]


Как мы видим, низкая точность показателя sitting связана с тем, что алгоритм часто путает данный класс с классом Laying( 211 раз алгоритм оказался неточным) 

## Выбор модели 3 (LogisticRegression)

## Опробуем алгоритм "из коробки"

In [109]:
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
model3.fit(train_X, train_y)

yhat = model3.predict(test_X)
yhat

/opt/python/envs/default/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [110]:

print(classification_report(test_y, yhat, target_names=target_names))

                    precision    recall  f1-score   support

           Walking       0.95      0.99      0.97       496
  Walking Upstairs       0.96      0.94      0.95       471
Walking Downstairs       0.99      0.96      0.97       420
           Sitting       0.96      0.88      0.92       491
          Standing       0.90      0.97      0.93       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



## Теперь попробуем поиграться с гиперпараметрами

In [75]:
from sklearn.linear_model import LogisticRegression
model3_best = LogisticRegression(
    C=1.0, #сила регуляризации. Меньшие значения указывают на более сильную регуляризацию
    class_weight=None, #определяет веса, относящиеся к каждому классу. Когда нет, все классы имеют единицу веса.
    dual=False, # не совсем понял за что отвечает, поэтому использовал как в default
    fit_intercept=True, #следует ли добавить константу (также известную как смещение или перехват) к функции принятия решения.
    intercept_scaling=1, #синтетический вес признака
    l1_ratio=None, #Useful only when the solver ‘liblinear’ is used
    max_iter=150, #максимальное количество итераций
    multi_class='ovr', 
    n_jobs=None, #целое число или None (по умолчанию), определяющее количество используемых параллельных процессов. None обычно означает использование одного ядра, а -1 означает использование всех доступных ядер.
    penalty='l2',  #определяет, есть ли регуляризация и какой подход использовать
    random_state=0, #какой генератор псевдослучайных чисел использовать.
    solver='liblinear', # сделал как в default, какой решатель использовать
    tol=0.0001, #  определяет допуск для остановки процедуры, как default установил
    verbose=0, #определяющее степень детализации для решателей 
   
)
model3_best.fit(train_X, train_y)

yhat = model3_best.predict(test_X)
yhat

In [111]:
print(classification_report(test_y, yhat, target_names=target_names))

                    precision    recall  f1-score   support

           Walking       0.95      0.99      0.97       496
  Walking Upstairs       0.96      0.94      0.95       471
Walking Downstairs       0.99      0.96      0.97       420
           Sitting       0.96      0.88      0.92       491
          Standing       0.90      0.97      0.93       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



## Вывод LogisticRegression
 Показатели precision и recall показывают, что алгоритм точно предсказал действия человека. F1 мера для всех действий человека близка к единице, что свидетельствует об оптимальности выбора алгоритма для нашей задачи. Возможно, это связано с тем, что алгоритм дает на выходе 1 или 0, избегая при этом неопределенности. Обычно логистическая регрессия используется для предсказания одного из двух вариантов(например принятие решения о выдаче кредита клиенту), однако в нашей задаче с множеством вариантов предсказания алгоритм отработал довольно точно.

При изменении гиперпараметров изменения были несущественны, но немного удалось улучшить показатели метрик

# Выбор модели 4 (SVM)

### Использование линейного ядра

In [112]:
from sklearn import svm
model4 = svm.SVC(kernel='linear') # сначала попробуем линейное ядро, подразумевая что наши данные линейно разделимы
model4.fit(train_X, train_y)

yhat = model4.predict(test_X)
yhat

In [113]:
print(confusion_matrix(test_y,yhat)) # матрица неточностей
print(" ")  # пропустим место для красоты
print(classification_report(test_y, yhat, target_names=target_names))

[[492   1   3   0   0   0]
 [ 18 451   2   0   0   0]
 [  4   6 410   0   0   0]
 [  0   2   0 435  54   0]
 [  0   0   0  16 516   0]
 [  0   0   0   0   0 537]]
 
                    precision    recall  f1-score   support

           Walking       0.96      0.99      0.97       496
  Walking Upstairs       0.98      0.96      0.97       471
Walking Downstairs       0.99      0.98      0.98       420
           Sitting       0.96      0.89      0.92       491
          Standing       0.91      0.97      0.94       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.96      2947
         macro avg       0.97      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



### Использование полиномиального ядра
Возможно, в нашем случае нелинейно разделимые данные,тогда  прямая линия не может использоваться в качестве границы принятия решения. Попробуем полиномиальное ядро

In [114]:

model4_polynom_degree_3 = svm.SVC(kernel='poly', degree= 3 )
model4_polynom_degree_3.fit(train_X, train_y)

yhat = model4_polynom_degree_3.predict(test_X)
yhat

In [115]:
print(confusion_matrix(test_y,yhat)) # матрица неточностей
print(" ")  # пропустим место для красоты
print(classification_report(test_y, yhat, target_names=target_names))

[[489   4   3   0   0   0]
 [ 23 447   1   0   0   0]
 [  7  18 395   0   0   0]
 [  0   2   0 439  50   0]
 [  0   0   0  16 516   0]
 [  0   0   0   0   0 537]]
 
                    precision    recall  f1-score   support

           Walking       0.94      0.99      0.96       496
  Walking Upstairs       0.95      0.95      0.95       471
Walking Downstairs       0.99      0.94      0.96       420
           Sitting       0.96      0.89      0.93       491
          Standing       0.91      0.97      0.94       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



### Попробуем повысить значение degree до 8

In [116]:
model4_polynom_degree_8 = svm.SVC(kernel='poly', degree= 8 )
model4_polynom_degree_8.fit(train_X, train_y)

yhat = model4_polynom_degree_8.predict(test_X)
yhat

In [93]:
print(confusion_matrix(test_y,yhat)) # матрица неточностей
print(" ")  # пропустим место для красоты
print(classification_report(test_y, yhat, target_names=target_names))

[[479   5  12   0   0   0]
 [ 26 441   4   0   0   0]
 [  7  17 396   0   0   0]
 [  0   2   0 449  39   1]
 [  0   0   0   8 524   0]
 [  0   0   0   0   0 537]]
 
                    precision    recall  f1-score   support

           Walking       0.94      0.97      0.95       496
  Walking Upstairs       0.95      0.94      0.94       471
Walking Downstairs       0.96      0.94      0.95       420
           Sitting       0.98      0.91      0.95       491
          Standing       0.93      0.98      0.96       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



## Ядро Гаусса

In [117]:
model4_rbf = svm.SVC(kernel='rbf')
model4_rbf.fit(train_X, train_y)

yhat = model4_rbf.predict(test_X)
yhat

In [118]:
print(confusion_matrix(test_y,yhat)) # матрица неточностей
print(" ")  # пропустим место для красоты
print(classification_report(test_y, yhat, target_names=target_names))

[[488   5   3   0   0   0]
 [ 20 451   0   0   0   0]
 [ 10  26 384   0   0   0]
 [  0   2   0 438  51   0]
 [  0   0   0  29 503   0]
 [  0   0   0   0   0 537]]
 
                    precision    recall  f1-score   support

           Walking       0.94      0.98      0.96       496
  Walking Upstairs       0.93      0.96      0.94       471
Walking Downstairs       0.99      0.91      0.95       420
           Sitting       0.94      0.89      0.91       491
          Standing       0.91      0.95      0.93       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.95      2947
         macro avg       0.95      0.95      0.95      2947
      weighted avg       0.95      0.95      0.95      2947



## Сигмоидное ядро

In [119]:
model4_sigmoid = svm.SVC(kernel='sigmoid')
model4_sigmoid.fit(train_X, train_y)

yhat = model4_sigmoid.predict(test_X)
yhat

In [121]:
print(confusion_matrix(test_y,yhat)) # матрица неточностей
print(" ")  # пропустим место для красоты
print(classification_report(test_y, yhat, target_names=target_names))

[[475   0  20   0   1   0]
 [  9 455   3   0   4   0]
 [ 81  36 303   0   0   0]
 [  4   1   0 263 223   0]
 [  4   0   0  25 503   0]
 [  0   0   0   0   0 537]]
 
                    precision    recall  f1-score   support

           Walking       0.83      0.96      0.89       496
  Walking Upstairs       0.92      0.97      0.94       471
Walking Downstairs       0.93      0.72      0.81       420
           Sitting       0.91      0.54      0.68       491
          Standing       0.69      0.95      0.80       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.86      2947
         macro avg       0.88      0.85      0.85      2947
      weighted avg       0.88      0.86      0.85      2947



## Вывод SVM
 Если мы сравним производительность разных типов ядер, мы ясно увидим, что сигмоидное ядро работает хуже всего(accuracy = 0.86). Это связано с тем, что сигмоидная функция возвращает два значения, 0 и 1, поэтому она больше подходит для задач двоичной классификации. Однако в нашем случае у нас было 6 выходных классов.

 SVM с линейным ядром отработал лучше всего, значит наши данные линейно разделимы. SVM с полиномиальным ядром отработали примерно одинаково хорошо (рассматривал степени 3 и 8), ядро Гаусса отработало чуть хуже.
 
 В целом, показатели precision и recall показывают, что алгоритм точно предсказал действия человека.

# Вывод по лабораторной работе:
Я ознакомился с различными методами классификации в рамках машинного обучения.
 Были рассмотрены и сравнены между собой по показателям (precision, recall, f1-score, accuracy) следующие методы: 
•	KneighborsClassifier
•	GaussianNB
•	LogisticRegression
•	SVM
Наиболее точным классификатаром в нашем случае оказался SVM и LogisticRegression, наименее - GaussianNB
Также я получил первый опыт по работе с Jupiter Notebook.